<a href="https://colab.research.google.com/github/oakleighw/TSEG33ColourisationProject/blob/main/Full_Implementation_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#bnsreenu(2021) Source Code: python_for_microscopists, 090a-autoencoder_colorize_V0.2.py
#Available at: https://github.com/bnsreenu/python_for_microscopists/blob/master/090a-autoencoder_colorize_V0.2.py
#Last Accessed: 29 April 2021

#Importing Libraries
from keras.layers import Conv2D, UpSampling2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import tensorflow as tf

In [ ]:
#Mount google drive
from google.colab import drive
drive.mount('/content/drive')
root_path = 'gdrive/My Drive/your_project_folder/'  #change dir to your project folder

Mounted at /content/drive


In [3]:
#Upload kaggle.json file (Config file obtained by:
#To use the Kaggle API, sign up for a Kaggle account at https://www.kaggle.com. 
#Then go to the 'Account' tab of your user profile (https://www.kaggle.com/<username>/account) and select 'Create API Token'. 
#This will trigger the download of kaggle.json, a file containing your API credentials.)
#https://github.com/Kaggle/kaggle-api
from google.colab import files
files.upload()  #this will prompt you to upload the kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"oakleighweekes","key":"6f0881bc48d942697a39ce2027159f6d"}'}

In [4]:
#Installs kaggle API from json
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json  # set permission

kaggle.json


In [5]:
#download natural dataset
#Prasun Roy (2018) Natural Images. Kaggle Dataset. Available at: https://www.kaggle.com/prasunroy/natural-images. Last Accessed: 03 May 2021
!kaggle datasets download -d prasunroy/natural-images -p /content/gdrive/My\ Drive/kaggle/natural

 98% 336M/342M [00:05<00:00, 95.4MB/s]
100% 342M/342M [00:05<00:00, 68.7MB/s]


In [6]:
#Creating a folder for dataset to unzip to
import os
os.chdir('/content/gdrive/My Drive/kaggle/natural')  #change dir
!unzip -q natural-images.zip -d natural-images/  #unzip data in natural-images/

In [7]:
#folder for all images
if not os.path.exists('/content/gdrive/My Drive/kaggle/natural/natural-images/allImages'):
    os.makedirs('/content/gdrive/My Drive/kaggle/natural/natural-images/allImages')

In [8]:
#put all images into one folder
import shutil

# --------------------------------------------------------
reorg_dir = "/content/gdrive/My Drive/kaggle/natural/natural-images/natural_images"
target_dir = "/content/gdrive/My Drive/kaggle/natural/natural-images/allImages" 
# ---------------------------------------------------------
for root, dirs, files in os.walk(reorg_dir):
    for name in files:
        subject = root+"/"+name
        n = 1; name_orig = name
        while os.path.exists(target_dir+"/"+name):
            name = "duplicate_"+str(n)+"_"+name_orig; n = n+1
        newfile = target_dir+"/"+name; shutil.copy(subject, newfile)

In [9]:
#shuffling image names
import os
from random import shuffle

dirname = r'/content/gdrive/My Drive/kaggle/natural/natural-images/allImages'

paths = [
    os.path.join(root, filename)
    for root, dirs, files in os.walk(dirname)
    for filename in files
    if filename.endswith('.jpg')
]
shuffle(paths)
randomData= paths[:6890]

['/content/gdrive/My Drive/kaggle/natural/natural-images/allImages/person_0253.jpg', '/content/gdrive/My Drive/kaggle/natural/natural-images/allImages/motorbike_0205.jpg', '/content/gdrive/My Drive/kaggle/natural/natural-images/allImages/motorbike_0563.jpg', '/content/gdrive/My Drive/kaggle/natural/natural-images/allImages/car_0168.jpg', '/content/gdrive/My Drive/kaggle/natural/natural-images/allImages/car_0962.jpg', '/content/gdrive/My Drive/kaggle/natural/natural-images/allImages/cat_0496.jpg', '/content/gdrive/My Drive/kaggle/natural/natural-images/allImages/person_0779.jpg', '/content/gdrive/My Drive/kaggle/natural/natural-images/allImages/dog_0192.jpg', '/content/gdrive/My Drive/kaggle/natural/natural-images/allImages/fruit_0868.jpg', '/content/gdrive/My Drive/kaggle/natural/natural-images/allImages/dog_0628.jpg', '/content/gdrive/My Drive/kaggle/natural/natural-images/allImages/cat_0064.jpg', '/content/gdrive/My Drive/kaggle/natural/natural-images/allImages/dog_0459.jpg', '/conte

In [10]:
#creating directory to contain shuffled images (train_datagen works with a nested directory)
if not os.path.exists('/content/gdrive/My Drive/kaggle/natural/natural-images/allImagesProcessed'):
    os.makedirs('/content/gdrive/My Drive/kaggle/natural/natural-images/allImagesProcessed')

if not os.path.exists('/content/gdrive/My Drive/kaggle/natural/natural-images/allImagesProcessed/resized'):
    os.makedirs('/content/gdrive/My Drive/kaggle/natural/natural-images/allImagesProcessed/resized')

In [11]:
#save shuffled images to folder
from PIL import Image

count = 0
for imagename in randomData:
    image = Image.open(randomData[count])
    image.save('/content/gdrive/My Drive/kaggle/natural/natural-images/allImagesProcessed/resized/image_'+str(count)+'.jpg')
    count+=1

In [12]:
path = '/content/gdrive/My Drive/kaggle/natural/natural-images/allImagesProcessed/'

#Normalize images - divide by 255
train_datagen = ImageDataGenerator(rescale=1. / 255)

#Resize images, if needed
train = train_datagen.flow_from_directory(path, 
                                          target_size=(256, 256), 
                                          batch_size=340, 
                                          class_mode=None)



Found 6890 images belonging to 1 classes.


In [13]:
#convert images to greyscale for training and splitting into training and validation sets
X =[]
Y =[]
for img in train[0]:
  try:
      lab = rgb2lab(img)
      X.append(lab[:,:,0]) 
      Y.append(lab[:,:,1:] / 128) #A and B values range from -127 to 128, 
      #so we divide the values by 128 to restrict values to between -1 and 1.
  except:
     print('error')
X = np.array(X)
Y = np.array(Y)
X = X.reshape(X.shape+(1,)) #dimensions to be the same for X and Y
print(X.shape)
print(Y.shape)

(340, 256, 256, 1)
(340, 256, 256, 2)


In [14]:
#model definitions

#Encoder

model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2, input_shape=(256, 256, 1)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3,3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))

#Decoder
#Decoder
#Note: For the last layer we use tanh instead of Relu. 
#This is because we are colorizing the image in this layer using 2 filters, A and B.
#A and B values range between -1 and 1 so tanh (or hyperbolic tangent) is used
#as it also has the range between -1 and 1. 
#Other functions go from 0 to 1.
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(Conv2D(16, (3,3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='adam', loss='mse' , metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 64)      640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 128)       147584    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 256)       295168    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 256)       590080    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 512)       1180160   
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 512)       2

In [15]:
#create folder for saving model files
if not os.path.exists('/content/gdrive/My Drive/kaggle/natural/models'):
    os.makedirs('/content/gdrive/My Drive/kaggle/natural/models')

In [16]:
#train model
model.fit(X,Y,validation_split=0.3, epochs=350, batch_size=48)

#save model
model_json = model.to_json()
with open('/content/gdrive/My Drive/kaggle/natural/models/model.json', "w") as json_file:
    json_file.write(model_json)
model.save_weights("/content/gdrive/My Drive/kaggle/natural/models/model.h5")

model.save('/content/gdrive/My Drive/kaggle/natural/models/colorize_autoencoder.model')


Epoch 1/350
5/5 [==============================] - 29s 2s/step - loss: 0.6209 - accuracy: 0.5008 - val_loss: 0.9292 - val_accuracy: 0.3800
Epoch 2/350
5/5 [==============================] - 1s 301ms/step - loss: 0.9256 - accuracy: 0.3737 - val_loss: 0.9283 - val_accuracy: 0.3791
Epoch 3/350
5/5 [==============================] - 1s 300ms/step - loss: 0.8050 - accuracy: 0.3852 - val_loss: 0.0178 - val_accuracy: 0.3848
Epoch 4/350
5/5 [==============================] - 1s 300ms/step - loss: 0.0157 - accuracy: 0.3846 - val_loss: 0.0155 - val_accuracy: 0.4240
Epoch 5/350
5/5 [==============================] - 1s 301ms/step - loss: 0.0157 - accuracy: 0.4539 - val_loss: 0.0148 - val_accuracy: 0.6050
Epoch 6/350
5/5 [==============================] - 1s 301ms/step - loss: 0.0142 - accuracy: 0.5937 - val_loss: 0.0145 - val_accuracy: 0.6018
Epoch 7/350
5/5 [==============================] - 1s 300ms/step - loss: 0.0147 - accuracy: 0.5947 - val_loss: 0.0139 - val_accuracy: 0.6090
Epoch 8/350
5/5

In [18]:
#test model
tf.keras.models.load_model(
    '/content/gdrive/My Drive/kaggle/natural/models/colorize_autoencoder.model',
    custom_objects=None,
    compile=True)
img1_color=[]
img1=img_to_array(load_img("/content/gdrive/My Drive/kaggle/natural/natural-images/allImages/car_0186.jpg"))

img1 = resize(img1 ,(256,256))
imsave("previous.png", img1) #saves original image 
img1_color.append(img1)
img1_color = np.array(img1_color, dtype=float)
img1_color = rgb2lab(1.0/255*img1_color)[:,:,:,0]
img1_color = img1_color.reshape(img1_color.shape+(1,))
output1 = model.predict(img1_color)
output1 = output1*128
result = np.zeros((256, 256, 3))
result[:,:,0] = img1_color[0][:,:,0]
result[:,:,1:] = output1[0]
imsave("result.png", lab2rgb(result)) #saves colourised test image